# Datascience project
## Goal :
The goal of this project is to create graph of characters from transcripts of popular Tv shows from https://transcripts.foreverdreaming.org.
Theses graph will be merge per seasons and compared to the others graph in order to find correlations with their caracteristics.

## How will it works ?
First, we will get the data : we will download the transcripts for each tv show and store it into HTML files. Next, we will analyze each of them and generate one graph per episode and merge theses graphs to get a graph for each season. We will finally compare all the graphs with their caracteristics and try to find some characters pattern.

## Get the data :
So we will request https://transcripts.foreverdreaming.org and get the links of the tv shows :

In [1]:
import os
import sys
from bs4 import BeautifulSoup
import requests
import pickle
import os
import time
import re
from datetime import datetime
from pathlib import Path

In [2]:
#Global variables
urlWebsite = "https://transcripts.foreverdreaming.org";
#We will define a fake agent to contourn the protection
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
#Interval between two http requests :
interval = 1;

In [3]:
#Important functions :

#Return the soup of the link
def get_soup(url):
    print(get_now_datetime_str() + " -> request : " + url);
    agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
    page = requests.get(url, headers=agent)
    soup = BeautifulSoup(page.content)
    return soup;

#Return the soup of a local file
def get_moms_soup(path):
    print(get_now_datetime_str() + " -> file : " + path);
    page = BeautifulSoup(open(path, encoding='utf8'), "html.parser", from_encoding="utf-8")
    return page;


#Initialize a void pickle
def init_pickle(path):
    write_pickle(path, []);

#Write a pickle
def write_pickle(path, array_pickle):
    pickle_out = open(path,"wb");
    pickle.dump(array_pickle, pickle_out);
    pickle_out.close();

#Read a pickle
def read_pickle(path):
    file_path = Path(path);
    if(file_path.exists()):
        pickle_in = open(path,"rb");
        data = pickle.load(pickle_in);
        pickle_in.close();
        return data;
    else:
        return [];
    
#Clean a string
def clean_str(dirty_str):
    return re.sub('[^a-zA-Z0-9 \n\.]', "", dirty_str);

#Find the next page or return False if there is no next page
def find_next_page(soup):
    pagination = soup.findAll('b', attrs={'class':'pagination'})
    if(len(pagination) == 0):
        return False;
    other_pages = pagination[0].findAll('a')
    for page in other_pages:
        if(page.getText() == "»"):
            return page.get('href');
    return False;

#Transform the soup into a list of link to episode transcript
def html_to_url_list_episode(soup):
    link_list = [];
    flag_info = False #True if the rows of info are passed.
    table = soup.findAll('table', attrs={'class' : 'tablebg'});
    rows = table[0].findAll('tr');
    for row in rows:
        if(not(flag_info)):
            row_episode = row.findAll('b', attrs={'class' : 'gensmall'});
            if(len(row_episode) != 0 and row_episode[0].getText() == "Episode"):
                flag_info = True;
        else:
            line = row.findAll('a')[0];
            row_link = urlWebsite + line.get('href')[1:];
            name =  clean_str(line.getText());
            row_name = name;
            link_list.append([row_name, row_link]);
    return link_list;

#Return a string containing the current time
def get_now_datetime_str():
    now = datetime.now();
    return now.strftime("%H:%M:%S");

#make a dir if not exist
def mk_dir_ifn_exist(path):
    if not os.path.exists(path):
        os.mkdir(path);
   
    
#Loop the requesting of the page until it succeed
def req_until_death(url):
    time.sleep(interval);
    agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'};
    request_done = False;
    while(not(request_done)):
        try:
            print(get_now_datetime_str() + " -> request : " + url);
            page = requests.get(url, headers=agent, timeout=10);
            if(type(page) == "ReadTimeout" ):
                continue;
            else:
                request_done = True;
                soup = BeautifulSoup(page.content);
            return soup;
        except requests.exceptions.RequestException as e:
            print(get_now_datetime_str() + " -> failure, retry !");
            time.sleep(60);

In [ ]:
#Create important folders
mk_dir_ifn_exist("data");
mk_dir_ifn_exist("pickle");
mk_dir_ifn_exist("pickle/series");
mk_dir_ifn_exist("pickle/graphs");

### Step 1 : Get the index of all the tv shows
WARNING : Do not execute the following code if you already have the following files : 
- pickle/tv_shows.pickle
- pickle/tv_show_list.pickle 
- pickle/list_made.pickle !

In [ ]:
#Let's make our soup
soup = req_until_death(urlWebsite);

In [ ]:
#Now, we have to find all the <a> link with the css class "subforum"
tv_shows = {};
tv_shows_list = [];
for link in soup.findAll('a', attrs={"class" : "subforum"}):
    name =  clean_str(link.getText());
    tv_shows[name] = urlWebsite + link.get('href')[1:];
    tv_shows_list.append(name);

In [ ]:
write_pickle("pickle/tv_shows.pickle", tv_shows); #Contain a dict with the tv show and their link to the index of their transcript
write_pickle("pickle/tv_shows_list.pickle", tv_shows_list); #Contain a list of the tv_show

for name in tv_shows_list:
    if not os.path.exists("data/" + name):
        os.mkdir("data/" + name);
    if not os.path.exists("pickle/" + name):
        os.mkdir("pickle/" + name);

END OF THE WARNING  
Let's load the list of our tv shows and the dict that contains the link to the transcript's listing pages :

### Step 2 : Create the index
Now, we will create an index per tv show, each containing all the links to all his episodes !

In [4]:
tv_shows = read_pickle("pickle/tv_shows.pickle");
tv_shows_list = read_pickle("pickle/tv_shows_list.pickle");
list_series_already_indexed = read_pickle("pickle/list_series_already_indexed.pickle")

In [ ]:
for serie in tv_shows_list:
    print(get_now_datetime_str() + " : " + serie)
    if(serie in list_series_already_indexed and read_pickle("pickle/" + serie + "/episode_list.pickle") != []):
        continue;
        
    else:
        url = tv_shows[serie];
        all_episode = [];
        next_page_exist = True;
        while(next_page_exist): #If there is another index page, get it !
            soup = req_until_death(url);
            next_page = find_next_page(soup);
            if(next_page == False):
                next_page_exist = False;
            else:
                url = urlWebsite + next_page[1:];
            all_episode = all_episode + html_to_url_list_episode(soup);
            
        write_pickle("pickle/" + serie + "/episode_list.pickle", all_episode);
        
    list_series_already_indexed.append(serie);
    write_pickle("pickle/list_series_already_indexed.pickle", list_series_already_indexed);

### Step 3 : Download transcripts
Now, we will download all the transcript, it will have an HTML format.

In [5]:
def dl_transcript_page(soup, name, path):
    body = soup.findAll('div', attrs={'class':'postbody'})[0]
    final_path = path + "/" + name + ".html";
    with open(final_path, "w", encoding="utf-8") as file:
        file.write(body.prettify())

Rewrite the pickle file "already dl" for each serie based on the HTML files saved in case where the pickle save had an error

In [ ]:
for serie in tv_shows_list:
    print(get_now_datetime_str() + " : " + serie);
    episodes = read_pickle("pickle/" + serie + "/episode_list.pickle");
    episode_already_dl = [];
    for episode in episodes:
        name = episode[0];
        path = "data/" + serie + "/" + name + ".html";
        file_path = Path(path);
        if(file_path.exists()):
            print(get_now_datetime_str() + " -> OK for : " + name);
            episode_already_dl.append(episode[0])
            write_pickle("pickle/" + serie + "/episode_already_dl.pickle",episode_already_dl);

Download each HTML file containing the transcript of the episode.

In [ ]:
for serie in tv_shows_list:
    print(get_now_datetime_str() + " : " + serie);
    episodes = read_pickle("pickle/" + serie + "/episode_list.pickle");
    episode_already_dl = read_pickle("pickle/" + serie + "/episode_already_dl.pickle");
    
    for episode in episodes:
        if(episode[0] in episode_already_dl):
            continue;
        
        episode_already_dl.append(episode[0])
        name = episode[0];
        url = episode[1];
        print(get_now_datetime_str() + " -> Reading : " + episode[0]);
        soup = req_until_death(url);
        dl_transcript_page(soup, name, "data/" + serie);
        write_pickle("pickle/" + serie + "/episode_already_dl.pickle",episode_already_dl);

### Step 4 : Building our data
First we will extract the name of the characters and building their relations. Next, we will clean our data by excluding the outliers (that are probably false positive result of the character detection). We will build our graph and take some metrics to compare them.

In [5]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

Extract characters and relations from the html :

In [6]:
#clean a string
def clean_string(string_to_clean):
    string_to_clean = string_to_clean.replace('\n', ' ');
    string_to_clean = string_to_clean.strip();
    string_to_clean = string_to_clean.lstrip();
    string_to_clean = string_to_clean.rstrip();
    return string_to_clean;

#Return all the words identified as name of a character
def id_characters(sentence):
    name_list = [];
    doc = nlp(sentence)
    for ent in doc.ents:
        if(ent.label_ == "PERSON"): #Get all the words identified as character's name
            name = ent.text.lower();
            name_list.append(clean_string(name));
    return name_list;
    
#Extract characters and their relations from an html file
def extract_charac_of_episode(file_path):
    soup = get_moms_soup(file_path);
    lines_html = soup.findAll("p");
    characters_list = {};
    relations_list = [];
    
    for dirty_line in lines_html:
        line = dirty_line.get_text();
        line = clean_string(line);
        characs = id_characters(line);
        
        for i in range(0, len(characs)):
            #Create or increase the weight of the characters
            if(not(characs[i] in characters_list)):
                characters_list[characs[i]] = 1;
            else:
                characters_list[characs[i]] = characters_list[characs[i]] + 1;
        
            if(i+1 != len(characs)):
                for j in range(i+1, len(characs)):
                    next_charac = False; #The current character is the last one detected
                else:
                    next_charac = characs[i+1];
                relation_found = False;
                #Create or increase the weight of the relations
                for x in range(0, len(relations_list)):
                    if((characs[i] == relations_list[x][0] and next_charac == relations_list[x][1]) or (characs[i] == relations_list[x][1] and next_charac == relations_list[x][0])):
                        relation_found = True;
                        relations_list[x][2] = relations_list[x][2] + 1;
                        break;
                if(not(relation_found)):
                    relations_list.append([characs[i], next_charac, 1])
                    
    return characters_list, relations_list;

def merge_characs_serie(final_array, add_array):
    for charac in add_array:
        if(not(charac in final_array)):
            final_array[charac] = 1;
        else:
            final_array[charac] = final_array[charac] + add_array[charac];
        
    return final_array;

def merge_relations_serie(final_array, add_array):
    for relation in add_array:
        charac_1 = relation[0];
        charac_2 = relation[1];
        weight = relation[2];
        
        relation_found = False;
        for i in range(0, len(final_array)):
            final_relation = final_array[i];
            if((charac_1 == final_relation[1] and charac_2 == final_relation[0]) or charac_1 == final_relation[0] and charac_2 == final_relation[1]):
                final_array[i][2] = final_array[i][2] + weight;
                relation_found = True;
                break;
        
        if(not(relation_found)):
            final_array.append([charac_1, charac_2, weight]);
            
    return final_array;

#Extract all the characters and their relations from a serie (each episode one by one and merge the results)
def extract_charac_of_serie(serie):
    print(get_now_datetime_str() + " -> extract begin : " + serie);
    list_episodes = read_pickle('pickle/' + serie + "/episode_already_dl.pickle");
    final_characters = {};
    final_relations = [];
    
    for episode in list_episodes:
        path_episode = "data/" + serie + "/" + episode + '.html';
        temp_chara, temp_rela = extract_charac_of_episode(path_episode);
        final_characters = merge_characs_serie(final_characters, temp_chara);
        final_relations = merge_relations_serie(final_relations, temp_rela);
    
    print(get_now_datetime_str() + " -> extract end");
    return final_characters, final_relations;
        
    

In [ ]:
#Extract the characters and their relations from each tv show
series = read_pickle('pickle/list_series_already_indexed.pickle')
for serie in series:
    if(read_pickle('pickle/' + serie + '/relations.pickle') != [] or read_pickle('pickle/' + serie + '/characters.pickle') != []):
        continue;
        
    characters, relations = extract_charac_of_serie(serie);
    write_pickle('pickle/' + serie + '/characters.pickle', characters);
    write_pickle('pickle/' + serie + '/relations.pickle', relations);

Now, we have our basic data, we have to clean them by remove the lowest outliers. Not the highest because they are probably the main characters.
We will also remove the characters only mentionned one time and their relations because they are probably errors.

In [7]:
import numpy as np
import pandas as pd

def clean_data(serie):
    print(get_now_datetime_str() + " -> cleaning : " + serie);
    characters = {};
    dirty_characters = [];
    relations = [];
    relations_extract = read_pickle('pickle/' + serie + '/relations.pickle');
    characters_extract = read_pickle('pickle/' + serie + '/characters.pickle');
    
    if(characters_extract != {}):
        #First supress the one-occurences
        for charac in characters_extract:
            if(characters_extract[charac] > 1):
                characters[charac] = characters_extract[charac];
            else:
                dirty_characters.append(charac);

        #Now, we remove the low outliers using IQR only with lower bound*1.5 :
        if(characters != {}):
            weights_characs = [];
            for charac in characters:
                weights_characs.append(characters[charac]);
            sorted(weights_characs);
            
            q1, q3= np.percentile(weights_characs,[25,75]);
            iqr = q3 - q1;
            lower_bound = q1 -(1.5 * iqr);
            new_characs = {};
            for charac in characters:
                if(characters[charac] < lower_bound):
                    dirty_characters.append(charac);
                else:
                    new_characs[charac] = characters[charac];
            characters = new_characs;
            
        #We remove the relations related to the removed characters
        for relation in relations_extract:
            if(relation[0] in dirty_characters or relation[1] in dirty_characters):
                continue;
            else:
                relations.append(relation); 
        
        #Check and replace if you found a more longest is declared for the same character
        replaced_names = [];
        while(True):
            replaced_character = False;

            for name in characters:
                for new_name in characters:
                    if((name in new_name) and (len(name) < len(new_name)) and (not(name in replaced_names))): #If one of the two name contain the other and is bigger
                        replaced_character = True;
                        weight = characters[new_name] + characters[name];
                        characters[new_name] = weight;
                        replaced_names.append(name);

                        #TODO : Replace relations by a Hashmap based on the names of the two characters
                        for relation in relations:  #Update the names in the relations
                            if(relation[0] == charac):
                                relation[0] = new_name;
                            if(relation[1] == charac):
                                relation[1] = new_name;
            if(not(replaced_character)):
                break;
                
        for name in replaced_names:   
            del characters[name];
                
    write_pickle('pickle/' + serie + '/characters_cleaned.pickle', characters);
    write_pickle('pickle/' + serie + '/relations_cleaned.pickle', relations);
    return characters, relations;

In [ ]:
series = read_pickle('pickle/list_series_already_indexed.pickle')
test = [];
for serie in series:
    if(read_pickle('pickle/' + serie + '/relations.pickle') == [] and read_pickle('pickle/' + serie + '/characters.pickle') == []):
        continue;
    characs, _ = clean_data(serie);

So, we now will create our graphs, one per serie :

In [21]:
import networkx as nx;

In [22]:
series = read_pickle('pickle/list_series_already_indexed.pickle')
for serie in series:    
    graph_serie = nx.Graph();
    relations = read_pickle('pickle/' + serie + '/relations_cleaned.pickle');
    characters = read_pickle('pickle/' + serie + '/characters_cleaned.pickle');
    for character in characters:
        graph_serie.add_node(character, weight=characters[character]);
    for relation in relations:
        graph_serie.add_edge(relation[0], relation[1], weight=relation[2]);
    write_pickle('pickle/' + serie + '/graph.pickle', graph_serie)
    
    

Now, we will analyze theses graph as if they was social network graph. In this case, the main metrics are metrics of connections, distributions and segmentations :
- Connections :
    - Homophily (assortativity)
    - Multiplexity (average degree of connectivity)
    - Network Closure (transivitive closure)
- Distributions :
    - Centrality (degree, closeness, etc.)
    - Density
    - Structural holes
- Segmentation
    - Clustering coefficient
    - Cohesion
    
So, we will first save our graph in picture and next analyze their caracteristics.

In [37]:
import matplotlib.pyplot as plt
series = read_pickle('pickle/list_series_already_indexed.pickle')
for serie in series:
    print(get_now_datetime_str() + " -> graph compute : " + serie);
    graph = read_pickle('pickle/' + serie + '/graph.pickle')
    graph_caracs = {};
    
    file_path = Path("./img/graph_" + serie + ".png");
    if(not(file_path.exists())):
        G = graph;
        fig = plt.figure(figsize=(12,12))
        ax = plt.subplot(111)
        ax.set_title(serie, fontsize=10)

        pos = nx.spring_layout(G)
        nx.draw(G, pos, node_size=1500, node_color='black', font_size=8, font_weight='bold')

        plt.tight_layout()
        plt.savefig("./img/graph_" + serie + ".png", format="PNG")
        plt.close();
    
    print(nx.numeric_assortativity_coefficient(graph, 'weight'))
    
    
    graph_caracs["homophily"] = nx.numeric_assortativity_coefficient(graph, 'weight');
    graph_caracs["multiplexy"] = nx.average_degree_connectivity(graph, weight='weight');
    graph_caracs["net_closure"] = nx.transitive_closure(graph);
    
    graph_caracs["centrality_degree"] = nx.degree_centrality(graph);
    graph_caracs["centrality_closeness"] = nx.closeness_centrality(graph);
    graph_caracs["centrality_eigvect"] = nx.eigenvector_centrality(graph, weight="weight");
    graph_caracs["density"] = nx.density(graph);
    graph_caracs["struct_holes"] = nx.constraint(graph, weight="weight");
    
    graph_caracs["avg_clus_coef"] = nx.average_clustering(graph, weight='weight');
    graph_caracs["min_clus_coef"] = min(nx.clustering(graph, weight='weight'));
    graph_caracs["max_clus_coef"] = max(nx.clustering(graph, weight='weight'));
    
    graph_caracs['cohesion'] = nx.k_components(graph);
    
    
    write_pickle('pickle/' + serie + '/graph_caracs.pickle', graph_caracs)
    break;

14:16:38 -> graph compute : 12 Monkeys


TypeError: '>' not supported between instances of 'int' and 'NoneType'